In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

df = pd.read_csv('./data/raw/compounds_final.csv', index_col=0)
df.head(1)

data = pd.DataFrame(df['SMILES'])
data.rename(columns={'SMILES': 'smiles'}, inplace=True)
data['label'] = np.nan
data.head()

# check if dir exists and create if not
if not os.path.exists('./data/kegg_data/raw'):
    os.makedirs('./data/kegg_data/raw')

if not os.path.exists('./data/kegg_data/processed'):
    os.makedirs('./data/kegg_data/processed')

# save data to csv
data.to_csv('./data/kegg_data/raw/data.csv', index=False)

In [13]:
from dataset import MoleculeDataset

kegg_dataset = MoleculeDataset(root='./data/kegg_data/', filename='data.csv', test=True)
print(f'Length of dataset: {len(kegg_dataset)}')

Length of dataset: 8591


In [14]:
from train import load_torch_model, predict

model = load_torch_model(feature_size=kegg_dataset[0].x.shape[1])

y = []
for i in tqdm(range(len(kegg_dataset))):
        
    y_true, y_pred = predict(model, kegg_dataset[i])
    y.append(y_pred[0])

df['toxic'] = y

  0%|          | 0/8591 [00:00<?, ?it/s]/home/WUR/g0012069/env/main/lib/python3.10/site-packages/torch_geometric/utils/scatter.py:93: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(f"The usage of `scatter(reduce='{reduce}')` "
  0%|          | 37/8591 [00:00<01:09, 122.42it/s]

100%|██████████| 8591/8591 [00:46<00:00, 186.17it/s]


In [18]:
# save df to csv
df.to_csv('./data/raw/compounds_final.csv')